<a href="https://colab.research.google.com/github/riskyhomo/Material-Discovery/blob/main/GANs_for_MD_500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np

In [18]:
df = pd.read_csv("/content/500.csv")
df

,Unnamed: 0,Unnamed: 0.1,Composition,MaterialId,Reduced Formula,Elements,NSites,Volume,Density,Point Group,...,Crystal System,Corrected Energy,Formation Energy Per Atom,Decomposition Energy Per Atom,Dimensionality Cheon,Bandgap,Is Train,Decomposition Energy Per Atom All,Decomposition Energy Per Atom MP,Decomposition Energy Per Atom MP OQMD
0,0,0,209231,0,39871,163207,10,302.9202,3.2834,10,...,2,-70.4155,-1.9058,-0.1625,5,0.0023,1.0,0.0,-0.1625,-0.0733
1,1,1,355560,1,220471,181188,16,430.2851,6.8040,8,...,5,-94.0804,-0.5004,-0.0137,3,0.0022,1.0,0.0,-0.0137,-0.0106
2,2,2,173457,2,143223,83526,36,828.6647,12.7292,8,...,5,-237.6016,-1.2960,-0.1195,3,inf,1.0,0.0,-0.1195,-0.1185
3,3,3,344245,3,364467,102592,17,407.2614,8.6776,0,...,5,-93.6931,-0.3064,-0.0837,3,0.0011,1.0,-0.0,-0.0837,-0.0772
4,4,4,254904,4,78813,176755,36,727.5283,6.9851,8,...,5,-254.0953,-0.8718,-0.0186,3,inf,1.0,-0.0,-0.0186,-0.0176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,495,249041,495,272536,191723,32,760.5079,12.0442,3,...,4,-224.3199,-0.8070,-0.0045,3,0.0008,1.0,0.0,-0.0045,-0.0045
496,496,496,355679,496,201679,130326,36,529.8488,7.1464,9,...,2,-318.7532,-3.2135,-0.0105,3,0.0075,1.0,0.0,-0.0105,-0.0103
497,497,497,205713,497,177779,142450,11,154.8750,9.1801,30,...,3,-90.3639,-0.7422,-0.1180,3,0.0020,1.0,0.0,-0.1180,-0.0564
498,498,498,293593,498,134196,83347,28,773.0204,4.7142,0,...,5,-173.5999,-1.5670,-0.0733,5,0.0007,1.0,0.0,-0.0733,-0.0280


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 22 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Unnamed: 0                             500 non-null    int64  
 1   Unnamed: 0.1                           500 non-null    int64  
 2   Composition                            500 non-null    int64  
 3   MaterialId                             500 non-null    int64  
 4   Reduced Formula                        500 non-null    int64  
 5   Elements                               500 non-null    int64  
 6   NSites                                 500 non-null    int64  
 7   Volume                                 500 non-null    float64
 8   Density                                500 non-null    float64
 9   Point Group                            500 non-null    int64  
 10  Space Group                            500 non-null    int64  
 11  Space 

In [22]:
df.shape

(500, 22)

In [24]:
latent_dim = 100

In [26]:
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, ReLU,LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Reshape

In [28]:
generator = Sequential([
    Dense( 16,input_dim = latent_dim),
    LeakyReLU(0.2),
    BatchNormalization(),
    Dense(128),
    LeakyReLU(0.2),
    BatchNormalization(),
    Dense(512),
    LeakyReLU(0.2),
    BatchNormalization()
])

In [30]:
generator.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 16)                1616      
                                                                 
 leaky_re_lu_9 (LeakyReLU)   (None, 16)                0         
                                                                 
 batch_normalization_6 (Bat  (None, 16)                64        
 chNormalization)                                                
                                                                 
 dense_11 (Dense)            (None, 128)               2176      
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 128)               0         
                                                                 
 batch_normalization_7 (Bat  (None, 128)               512       
 chNormalization)                                     

In [32]:
degenerator = Sequential([
    Input(shape=(512,1)),
    Dense(512, activation=LeakyReLU(0.2)),
    Dense(128, activation=LeakyReLU(0.2)),
    Dense(16, activation=LeakyReLU(0.2)),
    Dense(1, activation="sigmoid")
])

In [34]:
degenerator.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 512, 512)          1024      
                                                                 
 dense_18 (Dense)            (None, 512, 128)          65664     
                                                                 
 dense_19 (Dense)            (None, 512, 16)           2064      
                                                                 
 dense_20 (Dense)            (None, 512, 1)            17        
                                                                 
Total params: 68769 (268.63 KB)
Trainable params: 68769 (268.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
degenerator.trainable=False
gan_input = Input(shape=(latent_dim,))
gan_output = degenerator(generator(gan_input))
gan = Model(gan_input, gan_output)

degenerator.compile(loss="mean_squared_error", optimizer=Adam(learning_rate=0.01, beta_1=0.5))
gan.compile(loss="mean_squared_error", optimizer = Adam(learning_rate=0.01, beta_1=0.5))

In [38]:
gan.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential_3 (Sequential)   (None, 512)               72464     
                                                                 
 sequential_5 (Sequential)   (None, 512, 1)            68769     
                                                                 
Total params: 141233 (551.69 KB)
Trainable params: 71152 (277.94 KB)
Non-trainable params: 70081 (273.75 KB)
_________________________________________________________________


In [40]:
latent_dim = 100
input_shape = (df.shape[1], 1)
batch_size = 2048
epochs = 100


In [ ]:
from sklearn.preprocessing  import MinMaxScaler
scaler = MinMaxScaler()
composition= df["Composition"].to_numpy().reshape(-1,1)
compositions_scaled = scaler.fit_transform(composition)

In [51]:
for epoch in range(epochs):
    # Generate real and fake samples
    idx = np.random.randint(0, 99, size=batch_size)
    real_samples = compositions_scaled[idx]
    noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
    fake_samples = generator.predict(noise)
    fake_samples = np.mean(fake_samples, axis=1)
    fake_samples= np.expand_dims(fake_samples, axis=1)

    # Train the discriminator
    X = np.concatenate((real_samples, fake_samples), axis=0)
    y = np.concatenate((np.ones((batch_size, 1)), np.zeros((batch_size, 1))), axis=0)

    discriminator_loss = degenerator.train_on_batch(X, y)

    # Train the generator
    noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
    y_gan = np.ones(batch_size)
    generator_loss = gan.train_on_batch(noise, y_gan)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{epochs}, Discriminator Loss: {discriminator_loss}, Generator Loss: {generator_loss}")

64/64 [==============================] - 0s 2ms/step
Epoch 0/100, Discriminator Loss: 0.2505013942718506, Generator Loss: 0.2694104313850403
64/64 [==============================] - 0s 1ms/step
Epoch 10/100, Discriminator Loss: 0.250369668006897, Generator Loss: 0.2597614526748657
64/64 [==============================] - 0s 2ms/step
Epoch 20/100, Discriminator Loss: 0.25051718950271606, Generator Loss: 0.2507551908493042
64/64 [==============================] - 0s 1ms/step
Epoch 30/100, Discriminator Loss: 0.2505009174346924, Generator Loss: 0.25049588084220886
64/64 [==============================] - 0s 2ms/step
Epoch 40/100, Discriminator Loss: 0.25052395462989807, Generator Loss: 0.2505238652229309
64/64 [==============================] - 0s 2ms/step
Epoch 50/100, Discriminator Loss: 0.25052011013031006, Generator Loss: 0.25045502185821533
64/64 [==============================] - 0s 1ms/step
Epoch 60/100, Discriminator Loss: 0.25053104758262634, Generator Loss: 0.25042805075645447
6

In [94]:
num_materials = 10
noise = np.random.normal(0, 1, size=(num_materials, latent_dim))
new_compositions_encoded = generator.predict(noise)
new_compositions = scaler.inverse_transform(new_compositions_encoded)


#for composition in new_compositions:
    #composition_str = ' '.join(map(str, composition))
    #print(composition_str)

data = [{'composition': composition_str} for composition in composition_str]
df = pd.DataFrame(data)
print(df)

1/1 [==============================] - 0s 26ms/step
                                            composition
0     31010.01 22378.93 24340.578 23583.316 19136.29...
1     31010.01 22378.93 24340.578 23583.316 19136.29...
2     31010.01 22378.93 24340.578 23583.316 19136.29...
3     31010.01 22378.93 24340.578 23583.316 19136.29...
4     31010.01 22378.93 24340.578 23583.316 19136.29...
...                                                 ...
4994  31010.01 22378.93 24340.578 23583.316 19136.29...
4995  31010.01 22378.93 24340.578 23583.316 19136.29...
4996  31010.01 22378.93 24340.578 23583.316 19136.29...
4997  31010.01 22378.93 24340.578 23583.316 19136.29...
4998  31010.01 22378.93 24340.578 23583.316 19136.29...

[4999 rows x 1 columns]


AttributeError: 'numpy.ndarray' object has no attribute 'value_counts'

In [119]:
import numpy as np
import io

element_symbols = {
    0: 'H', 1: 'He', 2: 'Li', 3: 'Be', 4: 'B', 5: 'C', 6: 'N', 7: 'O', 8: 'F', 9: 'Ne',
    10: 'Na', 11: 'Mg', 12: 'Al', 13: 'Si', 14: 'P', 15: 'S', 16: 'Cl', 17: 'Ar', 18: 'K', 19: 'Ca',
    20: 'Sc', 21: 'Ti', 22: 'V', 23: 'Cr', 24: 'Mn', 25: 'Fe', 26: 'Co', 27: 'Ni', 28: 'Cu', 29: 'Zn',
    30: 'Ga', 31: 'Ge', 32: 'As', 33: 'Se', 34: 'Br', 35: 'Kr', 36: 'Rb', 37: 'Sr', 38: 'Y', 39: 'Zr',
    40: 'Nb', 41: 'Mo', 42: 'Tc', 43: 'Ru', 44: 'Rh', 45: 'Pd', 46: 'Ag', 47: 'Cd', 48: 'In', 49: 'Sn',
    50: 'Sb', 51: 'Te', 52: 'I', 53: 'Xe', 54: 'Cs', 55: 'Ba', 56: 'La', 57: 'Ce', 58: 'Pr', 59: 'Nd',
    60: 'Pm', 61: 'Sm', 62: 'Eu', 63: 'Gd', 64: 'Tb', 65: 'Dy', 66: 'Ho', 67: 'Er', 68: 'Tm', 69: 'Yb',
    70: 'Lu', 71: 'Hf', 72: 'Ta', 73: 'W', 74: 'Re', 75: 'Os', 76: 'Ir', 77: 'Pt', 78: 'Au', 79: 'Hg',
    80: 'Tl', 81: 'Pb', 82: 'Bi', 83: 'Po', 84: 'At', 85: 'Rn', 86: 'Fr', 87: 'Ra', 88: 'Ac', 89: 'Th',
    90: 'Pa', 91: 'U', 92: 'Np', 93: 'Pu', 94: 'Am', 95: 'Cm', 96: 'Bk', 97: 'Cf', 98: 'Es', 99: 'Fm', 100 :'Md', 101 :'Unn' , 102: 'No', 103 :'Lr', 104:'Rf', 105:'Db', 106:'Sg', 107:'Bh', 108:'Hs', 109:'Mt', 110:'Ds', 111:'Rg', 112:'Cn', 113:'Nh', 114:'Fl', 115:'Lv', 116:'Ts', 117:'Og'
}
num_materials = 10
noise = np.random.normal(0, 1, size=(num_materials, latent_dim))
new_compositions_encoded = generator.predict(noise)
#new_compositions_encoded = new_compositions_encoded.reshape(-1)
new_compositions_encoded = np.clip(new_compositions, composition.min(), composition.max())
decoded_formulas = safe_inverse_transform(label_encoder, new_compositions_encoded.astype(int), default_value='UNKNOWN')


def composition_to_formula(composition):
    formula = ''
    for i, percentage in enumerate(composition):
        if percentage > 0:
            symbol = element_symbols.get(i, '')
            formula += f'{symbol}{int(percentage)}'
    return formula



data = [{'composition': formula} for composition in formula]
df = pd.DataFrame(data)
df.to_csv("NEW.csv")

1/1 [==============================] - 0s 26ms/step


In [113]:
new_compositions_encoded

array([30274.395, 30274.395, 30274.395, ..., 30274.395, 30274.395,
       30274.395], dtype=float32)

In [121]:
def safe_inverse_transform(le, labels, default_value='UNKNOWN'):

    unique_labels = set(labels)


    if unique_labels.issubset(label_encoder.classes_):
        return label_encoder.inverse_transform(labels)
    else:

        seen_labels = [label for label in labels if label in label_encoder.classes_]
        unseen_labels = [label for label in labels if label not in label_encoder.classes_]


        transformed_seen_labels = label_encoder.inverse_transform(seen_labels)


        final_labels = []


        for label in labels:
            if label in seen_labels:

                index = seen_labels.index(label)

                final_labels.append(transformed_seen_labels[index])
            else:

                final_labels.append(default_value)

        return np.array(final_labels)


In [120]:
df234= pd.read_csv("/content/NEW.csv")
df234

,Unnamed: 0,composition
0,0,H29456He28115Li28052Be30442B22945C27271N29329O...
1,1,H29456He28115Li28052Be30442B22945C27271N29329O...
2,2,H29456He28115Li28052Be30442B22945C27271N29329O...
3,3,H29456He28115Li28052Be30442B22945C27271N29329O...
4,4,H29456He28115Li28052Be30442B22945C27271N29329O...
...,...,...
5536,5536,H29456He28115Li28052Be30442B22945C27271N29329O...
5537,5537,H29456He28115Li28052Be30442B22945C27271N29329O...
5538,5538,H29456He28115Li28052Be30442B22945C27271N29329O...
5539,5539,H29456He28115Li28052Be30442B22945C27271N29329O...
